In [1]:
# LSTM for international airline passengers problem with window regression framing
import numpy
import numpy as np
import keras
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.cross_validation import train_test_split
from keras.utils.vis_utils import plot_model

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [3]:
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset
# dataframe = read_csv('w_d_v.csv', usecols=[7], engine='python', skipfooter=3)
dataframe = read_csv('00.txt', usecols=[0], engine='python',skiprows=-1)
all_data = read_csv('all_data.csv', usecols=[7], engine='python', skipfooter=3)
dataset = dataframe.values
allData=all_data.values

In [4]:
look_back = 3
trainX, trainY = create_dataset(dataset, look_back)
AllX, AllY = create_dataset(allData, look_back)
trainY=numpy.reshape(trainY,(trainY.shape[0],-1))
AllY=numpy.reshape(AllY,(AllY.shape[0],-1))

In [5]:
encX = OneHotEncoder()
encX.fit(AllX)
encY = OneHotEncoder()
encY.fit(AllY)

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [6]:
trainX_one=encX.transform(trainX).toarray()
train_X=numpy.reshape(trainX_one,(trainX_one.shape[0],look_back,-1))
train_Y=encY.transform(trainY).toarray()

In [7]:
a_train, a_test, b_train, b_test = train_test_split(train_X, train_Y, test_size=0.1, random_state=42)

In [8]:
model = Sequential()
# model.add(Embedding(max_features, output_dim=256))
model.add(LSTM(512,return_sequences=True,
               input_shape=(3, a_train.shape[2]),name="lstm"))  # returns a sequence of vectors of dimension 32
model.add(LSTM(300))  # return a single vector of dimension 32
model.add(Dropout(0.2))
model.add(Dense(250,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(a_train.shape[2],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
plot_model(model, to_file='lstm.png')

In [76]:
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model

# x = Embedding(output_dim=512, input_dim=10000, input_length=100)(main_input)
# lstm_out = LSTM(512,input_shape=(3, a_train.shape[2]))(main_input)

t_input = Sequential()
t_input.add(LSTM(512,return_sequences=True,input_shape=(3, a_train.shape[2]))) 

out = Sequential()
out.add(Dense(a_train.shape[2],activation='softmax',name="out"))

model = Model(inputs=[t_input], outputs=[out])


ValueError: The first layer in a Sequential model must get an `input_shape` or `batch_input_shape` argument.

In [73]:
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model

main_input = Input(shape=(100,), dtype='int32', name='main_input')
x = Embedding(output_dim=512, input_dim=10000, input_length=100)(main_input)
lstm_out = LSTM(32)(x)

# main_input = Sequential()
# main_input.add(LSTM(512,return_sequences=True,input_shape=(3, a_train.shape[2]))) 

pattern_output = Dense(1, activation='sigmoid', name='pattern_output')(lstm_out)

UserId_input = Input(shape=(a_train.shape[0],), name='UserId_input')
x = keras.layers.concatenate([UserId_input, pattern_output])

# We stack a deep densely-connected network on top
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)

# And finally we add the main logistic regression layer
main_output = Dense(a_train.shape[2], activation='softmax', name='main_output')(x)

model = Model(inputs=[main_input, UserId_input], outputs=[main_output, pattern_output])
model.compile(optimizer='adam',
              loss={'main_output': 'categorical_crossentropy', 'pattern_output': 'categorical_crossentropy'},
              metrics=['accuracy'])
plot_model(model, to_file='New_lstm.png')

In [66]:
k=np.zeros(a_train.shape[0])
k=k.reshape(-1,1)

In [67]:
model.fit([a_train,k], [b_train,b_train], epochs=100, batch_size=16, verbose=2, validation_data=(a_test, b_test))

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 2 array(s), but instead got the following list of 1 arrays: [array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
  ...

In [11]:
#batchsize批尺寸
model.fit(a_train, b_train, epochs=100, batch_size=16, verbose=2, validation_data=(a_test, b_test))

Train on 226 samples, validate on 26 samples
Epoch 1/100
 - 1s - loss: 2.4579 - acc: 0.3363 - val_loss: 2.8475 - val_acc: 0.3846
Epoch 2/100
 - 1s - loss: 2.4008 - acc: 0.3451 - val_loss: 2.8954 - val_acc: 0.3846
Epoch 3/100
 - 1s - loss: 2.3854 - acc: 0.3274 - val_loss: 2.8316 - val_acc: 0.3846
Epoch 4/100
 - 1s - loss: 2.4072 - acc: 0.3186 - val_loss: 2.8694 - val_acc: 0.3846
Epoch 5/100
 - 1s - loss: 2.3989 - acc: 0.3451 - val_loss: 2.9294 - val_acc: 0.3846
Epoch 6/100
 - 1s - loss: 2.4000 - acc: 0.2699 - val_loss: 2.8634 - val_acc: 0.4231
Epoch 7/100
 - 1s - loss: 2.3390 - acc: 0.3363 - val_loss: 3.0185 - val_acc: 0.3846
Epoch 8/100
 - 1s - loss: 2.3124 - acc: 0.3407 - val_loss: 3.0019 - val_acc: 0.3462
Epoch 9/100
 - 1s - loss: 2.3052 - acc: 0.3319 - val_loss: 3.0947 - val_acc: 0.4231
Epoch 10/100
 - 1s - loss: 2.2796 - acc: 0.3407 - val_loss: 3.0116 - val_acc: 0.3462
Epoch 11/100
 - 1s - loss: 2.2473 - acc: 0.3363 - val_loss: 3.0202 - val_acc: 0.3462
Epoch 12/100
 - 1s - loss: 2.

Epoch 97/100
 - 1s - loss: 0.5909 - acc: 0.7655 - val_loss: 4.9425 - val_acc: 0.1923
Epoch 98/100
 - 1s - loss: 0.6218 - acc: 0.7788 - val_loss: 5.0224 - val_acc: 0.1923
Epoch 99/100
 - 1s - loss: 0.6314 - acc: 0.7566 - val_loss: 5.0488 - val_acc: 0.2308
Epoch 100/100
 - 1s - loss: 0.5999 - acc: 0.7434 - val_loss: 5.0392 - val_acc: 0.1923


In [ ]:
model.evaluate(train_X, train_Y, batch_size=64, verbose=2, sample_weight=None)

In [ ]:
trainPredict = model.predict(train_X)
D=np.argmax(train_Y,axis = 1)
E=np.argmax(trainPredict,axis = 1)

In [ ]:
print(D)
print(E)

In [ ]:
A=0 #total number of right
for i,t in enumerate(E):
    if D[i]==t :
        A=A+1
print(A/D.shape[0])